In [12]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
import json
import os
from scipy.stats import ttest_rel

In [13]:
plt.rcParams['font.sans-serif'] = 'Times New Roman'

In [14]:
def read_json_list(json_file):
    if not (os.path.exists(json_file) and os.path.isfile(json_file)):
        with open(json_file, 'w') as f:
            f.write('[]')
    with open(json_file, 'r', encoding='utf-8') as f:
        try:
            lst = json.load(f)
        except:
            lst = []
    return lst

In [15]:
# From classical method
angle = read_json_list('./angle.dat')  # Angle
dists = read_json_list('./dist.dat')  # Spacing
length = read_json_list('./length.dat')  # Density

# From Bezier curve
curv = read_json_list('./curv.dat') # Curvature
num_conn = read_json_list('./num_conn.dat')  # Connected Compnents
winding = read_json_list('./winding.dat') # Winding Degree

person = 11

In [16]:
total_winding = [[[]],[[]]]
mean_winding = [[[]],[[]]]
average_curv = [[[]],[[]]]

for i in range(2):    # Type
    for j in range(1, len(winding[i])):     # Person
        total_winding[i].append([])
        mean_winding[i].append([])
        for k in range(len(winding[i][j])):      # Character
            total_winding[i][j].append(np.sum(winding[i][j][k]))
            mean_winding[i][j].append(np.mean(winding[i][j][k]))
    for j in range(1, len(curv[i])):
        average_curv[i].append([])
        for k in range(len(curv[i][j])):
            tmp_cur = 0
            tmp_len = 0
            for l in range(len(curv[i][j][k])):     # Curve
                tmp_cur += curv[i][j][k][l][0]
                tmp_len += curv[i][j][k][l][1]
            average_curv[i][j].append(tmp_cur/tmp_len)

In [17]:
all_total_winding = [[],[]]
all_mean_winding = [[],[]]
all_average_curv = [[],[]]
all_num_conn = [[],[]]
for i in range(2):
    for j in range(1, person+1):
        all_total_winding[i] = all_total_winding[i] + total_winding[i][j]
        all_mean_winding[i] = all_mean_winding[i] + mean_winding[i][j]
        all_average_curv[i] = all_average_curv[i] + average_curv[i][j]
        all_num_conn[i] = all_num_conn[i] + num_conn[i][j]

In [18]:
pvalue = []

pvalue.append(ttest_rel(angle[0],angle[1])[1])
pvalue.append(ttest_rel(length[0],length[1])[1])
pvalue.append(scipy.stats.wilcoxon(dists[0], dists[1], zero_method='wilcox', correction=False)[1])
pvalue.append(scipy.stats.wilcoxon(all_num_conn[0], all_num_conn[1], zero_method='wilcox', correction=False)[1])
pvalue.append(ttest_rel(all_total_winding[0],all_total_winding[1])[1])
pvalue.append(ttest_rel(all_average_curv[0],all_average_curv[1])[1])

In [19]:
with open('control_pvalue.dat', 'w') as f:
    json.dump(pvalue, f)

In [20]:
pvalue

[0.07685075846460883,
 0.2558432465085566,
 0.1814111579693004,
 0.23888830891117097,
 0.08559811407635104,
 0.985704725471767]